In [3]:
from z3 import *

# 1. Выражения, сорта и декларации

#### 1.1 Существует ли целое число, которое кратно 7 и 9, а также больше 0?

In [30]:
# Доказательство наличия решения

# целое число
x = Int("x")

# Условие
condition = And(x % 7 == 0, x % 9 == 0, x > 0)

print ("is expression: ", is_expr(condition))
print ("is application:", is_app(condition))
print ("decl:          ", condition.decl())
print ("num args:      ", condition.num_args())

# Доказываем что есть число, которое выполняет условия задачи (Квантор существования)
theorem = Exists( x, condition )

prove(theorem)

is expression:  True
is application: True
decl:           And
num args:       3
proved


In [14]:
# Решение

# Создаем солвер
solver = Solver()

# Добавляем условия в солвер
solver.add(condition)

# Решение задачи
result = solver.check()

if result == sat:
    model = solver.model()
    solution_x = model[x]
    print("Решение: x =", solution_x)
else:
    print("Решение не найдено. Нет целого числа, которое кратно и 7, и 9, и больше 0.")

Решение: x = 63


#### 1.2 Все числа, которые делятся на 4, также делятся на 2 (проверим и найдем решение для числа большего 4).

In [35]:
# Доказательство наличия решения

# целое число
x = Int("x")

# Условие
condition = And(x % 4 == 0, x > 4)

print ("is expression: ", is_expr(condition))
print ("is application:", is_app(condition))
print ("decl:          ", condition.decl())
print ("num args:      ", condition.num_args())

# Доказываем что все числа выполняет условия задачи (Квантор всеобщности)
theorem = ForAll( x, Implies(condition, x % 2 == 0) )

prove(theorem)

is expression:  True
is application: True
decl:           And
num args:       2
proved


In [36]:
# Решение

# Создаем солвер
solver = Solver()

# Добавляем условия в солвер
solver.add(condition)

# Решение задачи
result = solver.check()

if result == sat:
    model = solver.model()
    solution_x = model[x]
    print("Решение: x =", solution_x)
else:
    print("Решение не найдено. Нет целого числа, которое кратно и 7, и 9, и больше 0.")

Решение: x = 8


# 2. Массивы

### 2.1 Теорема: Если в массиве A все элементы равны нулю, то сумма элементов этого массива также равна нулю.

In [137]:
# Создание массива целых чисел
array_size = 5
A = Array('A', IntSort(), IntSort())

# Добавление ограничений, что все элементы равны нулю
constraints = [A[i] == 0 for i in range(array_size)]

# Добавление ограничения, что сумма элементов равна нулю
constraints.append(Sum([Select(A, i) for i in range(array_size)]) == 0)

# Доказательство
theorem = Implies(And(constraints), True)
prove(theorem)

proved


In [75]:
# Проверяю обратное
theorem = Implies(And(constraints), False)
prove(theorem)

counterexample
[A = K(Int, 0)]


### 2.2 Теорема: Если в массиве A все элементы являются четными числами, то их сумма также является четным числом.

In [136]:
# Создание массива целых чисел
array_size = 5
A = Array('A', IntSort(), IntSort())

# Добавление ограничений, что все элементы массива четные
constraints = [A[i] % 2 == 0 for i in range(array_size)]

# Добавление ограничения, что сумма элементов массива четная
constraints.append(Sum([Select(A, i) for i in range(array_size)]) % 2 == 0)

# Доказательство
theorem = Implies(And(constraints), True)
prove(theorem)

proved


In [81]:
# Проверяю обратное
theorem = Implies(And(constraints), False)
prove(theorem)

counterexample
[A = K(Int, 0)]


# 3. Кванторы

### 3.1 Теорема: Для любых двух целых чисел a и b, если a больше или равно нулю и b больше или равно нулю, то их сумма a + b также больше или равна нулю.

In [133]:
# Создание целых переменных a и b
a = Int('a')
b = Int('b')

# Ограничение
condition = And(a >= 0, b >= 0)

# Доказательство
theorem = ForAll([a, b], Implies(condition, a + b >= 0))
prove(theorem)

proved


In [134]:
# Ложное условие
theorem = ForAll([a, b], Implies(condition, a + b < 0))
prove(theorem)

counterexample
[]


In [116]:
# Решение

# Создание солвера
solver = Solver()

# Добавление ограничения в солвер
solver.add(condition)

# Решение задачи
result = solver.check()

if result == sat:
    model = solver.model()
    solution_a = model[a]
    solution_b = model[b]
    print("Решение: a =", solution_a, "b =", solution_b)
else:
    print("Решение не найдено. Условие не выполнено.")

Решение: a = 0 b = 0


### 3.2 Существует целое число x, которое удовлетворяет условию `x^2 - 4x + 4 = 0 и x > 1`.

In [132]:
# Создание целой переменной x
x = Int('x')

# Условие задачи
condition = And(x**2 - 4 * x + 4 == 0, x > 1)

# Доказательство
theorem = Exists(x, condition)

prove(theorem)

proved


In [131]:
# Решение

# Создание солвера
solver = Solver()

# Добавление ограничения в солвер
solver.add(condition)

# Решение задачи
result = solver.check()

if result == sat:
    model = solver.model()
    print("Решение: x =", model[x])
else:
    print("Решение не найдено. Условие не выполнено.")

Решение: x = 2


# 4. Оптимизация

### 4.1 Найти максимальный элемент массива

In [13]:
# Создание массива целых чисел
array_size = 5  # Размер массива
A = [Int(f'A{i}') for i in range(array_size)]

# Задание конкретных значений для элементов массива
A_values = [3, 8, 1, 12, 5]
constraints = [A[i] == A_values[i] for i in range(array_size)]

# Создание переменной для хранения максимального элемента
max_element = Int('max_element')

# Ограничение, что каждый элемент массива меньше или равен максимальному элементу
constraints += [A[i] <= max_element for i in range(array_size)]

# Создание объекта оптимизации
opt = Optimize()

# Добавление ограничений
for constraint in constraints:
    opt.add(constraint)

# Добавление цели для максимизации максимального элемента
opt.maximize(max_element)

# Решение задачи оптимизации
result = opt.check()

if result == sat:
    model = opt.model()
    max_value = model[max_element].as_long()
    print("Максимальный элемент в массиве:", max_value)
else:
    print("Решение не найдено.")


Максимальный элемент в массиве: 12


### 4.2 Минимизировать функцию f(x, y) = 2x + 3y при ограничениях:

```
x > 0
y > 0
x + y <= 10
2x - y <= 2
```
Найти значения x и y, которые минимизируют функцию `f(x, y)` при указанных ограничениях.

In [5]:
# Определение функции f(x, y)
f = Function('f', IntSort(), IntSort(), IntSort())

# Создание объекта оптимизации
opt = Optimize()

# Добавление ограничений
x = Int('x')
y = Int('y')
opt.add(x > 0)
opt.add(y > 0)
opt.add(x + y <= 10)
opt.add(2 * x - y <= 2)

# Добавление цели для минимизации функции f(x, y)
opt.minimize(f(x, y))

# Установка функции f(x, y) как 2x + 3y
opt.add(f(x, y) == 2 * x + 3 * y)

# Решение
result = opt.check()

if result == sat:
    model = opt.model()
    min_x = model[x].as_long()
    min_y = model[y].as_long()
    min_f_x_y = model.evaluate(f(x, y), model_completion=True).as_long()
    print("Минимальное значение функции f(x, y) =", min_f_x_y)
    print("x =", min_x)
    print("y =", min_y)
else:
    print("Решение не найдено.")


Минимальное значение функции f(x, y) = 11
x = 1
y = 3


# 5. Множественные солверы

### 5.1 Поиск альтернативных решений
С помощью использования нескольких солверов можно находить альтернативные решения. 
Тут стратегия заключается в том, что с помощью первого солвера находим первое решение. Далее создаем новых солвер, в который передаем изменение ограничения, чтобы найти альтернативное решение.

Например, нужно найти два целых числа x и y, таких что `x + y = 10`

In [6]:
# Создание переменных x и y
x = Int('x')
y = Int('y')

# Создание стандартного солвера
solver = Solver()

# Ограничение
constraint = x + y == 10

# Добавление ограничения
solver.add(constraint)

# Попытка найти первое решение
result = solver.check()
if result == sat:
    model = solver.model()
    x_value = model[x].as_long()
    y_value = model[y].as_long()
    print("Первое решение:")
    print(f"x = {x_value}")
    print(f"y = {y_value}")

# Создание нового солвера
new_solver = Solver()
# Изменение ограничения, чтобы найти альтернативное решение
new_constraint = And(x != x_value, y != y_value)
new_solver.add(constraint)
new_solver.add(new_constraint)

# Попытка найти альтернативное решение
result = new_solver.check()
if result == sat:
    model = new_solver.model()
    x_value = model[x].as_long()
    y_value = model[y].as_long()
    print()
    print("Альтернативное решение:")
    print(f"x = {x_value}")
    print(f"y = {y_value}")
else:
    print("Альтернативное решение не найдено.")


Первое решение:
x = 10
y = 0

Альтернативное решение:
x = 9
y = 1


### 5.2 Проверка устойчивости решения
Для проверки устойчивости решения можно использовать несколько разных солверов с разными стратегиями.
В данном примере исполуем SMT Solver и Bit-Vector Solver для нахождения двух целых чисел x и y, таких что `x + y = 15` и `x > 0` и `y > 0`.
Если солверы выдают одинаковый результат, то решение можно признать устойчивым.

In [10]:
# Создание контекста Z3 для SMT Solver
ctx_smt = Context()
# Создание контекста Z3 для Bit-Vector Solver
ctx_bitvector = Context()

# Создание переменных x и y для SMT Solver
x_smt = Int('x', ctx=ctx_smt)
y_smt = Int('y', ctx=ctx_smt)

# Создание переменных x и y для Bit-Vector Solver
x_bitvector = BitVec('x', 32, ctx=ctx_bitvector)  # 32-битные битовые векторы
y_bitvector = BitVec('y', 32, ctx=ctx_bitvector)

# Ограничение для SMT Solver: x + y = 15
constraint_smt = And(x_smt + y_smt == 15, x_smt > 0, y_smt > 0)

# Ограничение для Bit-Vector Solver: x + y = 15
constraint_bitvector = And(x_bitvector + y_bitvector == 15, x_bitvector > 0, y_bitvector > 0)

# Создание SMT Solver
solver_smt = Solver(ctx=ctx_smt)
solver_smt.add(constraint_smt)

# Создание Bit-Vector Solver
solver_bitvector = Solver(ctx=ctx_bitvector)
solver_bitvector.add(constraint_bitvector)

# Решение задачи с обоими солверами
result_smt = solver_smt.check()
result_bitvector = solver_bitvector.check()

# Вывод результатов
if result_smt == sat:
    model_smt = solver_smt.model()
    x_smt_value = model_smt[x_smt].as_long()
    y_smt_value = model_smt[y_smt].as_long()
    print("Решение SMT Solver:")
    print(f"x = {x_smt_value}")
    print(f"y = {y_smt_value}")
else:
    print("Решение SMT Solver не найдено.")

if result_bitvector == sat:
    model_bitvector = solver_bitvector.model()
    x_bitvector_value = model_bitvector[x_bitvector].as_long()
    y_bitvector_value = model_bitvector[y_bitvector].as_long()
    print()
    print("Решение Bit-Vector Solver:")
    print(f"x = {x_bitvector_value}")
    print(f"y = {y_bitvector_value}")
else:
    print("Решение Bit-Vector Solver не найдено.")


Решение SMT Solver:
x = 14
y = 1

Решение Bit-Vector Solver:
x = 14
y = 1
